# Configuration

RadiObject provides a unified configuration system with two levels of access:

| Prefix | Purpose | Functions |
|--------|---------|----------|
| `get_radiobject_config` / `reset_radiobject_config` | RadiObject configuration | `get_radiobject_config()`, `reset_radiobject_config()` |
| `get_tiledb_*` | TileDB internals | `get_tiledb_ctx()`, `get_tiledb_config()` |

**Key concepts:**
- **Write settings** (tile, compression, orientation) are **immutable after ingestion**
- **Read settings** (memory, concurrency) affect all read operations
- **S3 settings** configure cloud storage access

This notebook covers:
- Basic configuration with `configure()`
- Inspecting configuration with `get_radiobject_config()`
- Resetting configuration with `reset_radiobject_config()`
- Accessing TileDB internals with `get_tiledb_ctx()` and `get_tiledb_config()`
- Tile orientation strategies
- Compression options

**Prerequisites:** [01_radi_object.ipynb](./01_radi_object.ipynb)

## API Overview

```python
from radiobject import (
    # Configuration function
    configure,
    
    # RadiObject config
    get_radiobject_config,    # Get current RadiObject configuration
    reset_radiobject_config,  # Reset configuration to defaults
    
    # TileDB internals
    get_tiledb_ctx,     # Get TileDB context (lazy, cached)
    get_tiledb_config,  # Get raw TileDB Config object
    
    # Configuration classes
    WriteConfig, ReadConfig, S3Config,
    TileConfig, CompressionConfig, OrientationConfig,
    
    # Enums
    SliceOrientation, Compressor,
)
```

In [ ]:
import os
import shutil
import tempfile
import time
from pathlib import Path

import numpy as np

from radiobject import (
    CompressionConfig,
    Compressor,
    ReadConfig,
    S3Config,
    SliceOrientation,
    TileConfig,
    Volume,
    WriteConfig,
    configure,
    get_radiobject_config,
    get_tiledb_config,
    get_tiledb_ctx,
    reset_radiobject_config,
)

TEMP_DIR = tempfile.mkdtemp(prefix="config_tutorial_")
print(f"Working directory: {TEMP_DIR}")

## Basic Configuration

Use `configure()` to set RadiObject configuration. Settings are organized into three categories:

| Category | Class | Affects |
|----------|-------|--------|
| `write=` | `WriteConfig` | New arrays only (immutable after creation) |
| `read=` | `ReadConfig` | All read operations |
| `s3=` | `S3Config` | Cloud storage access |

In [2]:
# Configure for axial slice access with ZSTD compression
configure(
    write=WriteConfig(
        tile=TileConfig(orientation=SliceOrientation.AXIAL),
        compression=CompressionConfig(algorithm=Compressor.ZSTD, level=3),
    )
)

# View current configuration using get_radiobject_config()
config = get_radiobject_config()
print(f"Tile orientation: {config.write.tile.orientation}")
print(f"Compression: {config.write.compression.algorithm}, level={config.write.compression.level}")

Tile orientation: SliceOrientation.AXIAL
Compression: Compressor.ZSTD, level=3


## Inspecting Configuration: `get_radiobject_config()`

`get_radiobject_config()` returns the current `RadiObjectConfig` object with all settings:

In [3]:
config = get_radiobject_config()

# Explore the configuration structure
print("=== Write Settings (immutable after ingestion) ===")
print(f"  Tile orientation: {config.write.tile.orientation.value}")
print(
    f"  Compression: {config.write.compression.algorithm.value} (level {config.write.compression.level})"
)
print(f"  Auto-detect orientation: {config.write.orientation.auto_detect}")
print(f"  Reorient on load: {config.write.orientation.reorient_on_load}")

print("\n=== Read Settings (affects all reads) ===")
print(f"  Memory budget: {config.read.memory_budget_mb} MB")
print(f"  Concurrency: {config.read.concurrency} threads")
print(f"  Max workers: {config.read.max_workers}")

print("\n=== S3 Settings ===")
print(f"  Region: {config.s3.region}")
print(f"  Max parallel ops: {config.s3.max_parallel_ops}")
print(f"  Multipart size: {config.s3.multipart_part_size_mb} MB")

=== Write Settings (immutable after ingestion) ===
  Tile orientation: axial
  Compression: zstd (level 3)
  Auto-detect orientation: True
  Reorient on load: False

=== Read Settings (affects all reads) ===
  Memory budget: 1024 MB
  Concurrency: 4 threads
  Max workers: 4

=== S3 Settings ===
  Region: us-east-1
  Max parallel ops: 8
  Multipart size: 50 MB


## Resetting Configuration: `reset_radiobject_config()`

`reset_radiobject_config()` restores all settings to defaults and clears the cached TileDB context:

In [4]:
# Make some changes
configure(
    write=WriteConfig(tile=TileConfig(orientation=SliceOrientation.ISOTROPIC)),
    read=ReadConfig(memory_budget_mb=2048, concurrency=8),
)
print(
    f"Before reset: tile={get_radiobject_config().write.tile.orientation.value}, memory={get_radiobject_config().read.memory_budget_mb}MB"
)

# Reset to defaults
reset_radiobject_config()
print(
    f"After reset:  tile={get_radiobject_config().write.tile.orientation.value}, memory={get_radiobject_config().read.memory_budget_mb}MB"
)

Before reset: tile=isotropic, memory=2048MB
After reset:  tile=axial, memory=1024MB


## TileDB Internals: `get_tiledb_ctx()` and `get_tiledb_config()`

For advanced use cases, access the underlying TileDB objects:

| Function | Returns | Use Case |
|----------|---------|----------|
| `get_tiledb_ctx()` | `tiledb.Ctx` | Pass to TileDB operations, inspect thread pools |
| `get_tiledb_config()` | `tiledb.Config` | Inspect/modify raw TileDB parameters |

In [5]:
# Get the TileDB context (lazily created, cached)
ctx = get_tiledb_ctx()
print(f"TileDB Context: {type(ctx).__name__}")

# The context is cached - same object on repeated calls
ctx2 = get_tiledb_ctx()
print(f"Same context object: {ctx is ctx2}")

TileDB Context: Ctx
Same context object: True


In [6]:
# Get the raw TileDB Config object
tdb_config = get_tiledb_config()

# Inspect TileDB parameters directly
print("=== TileDB Configuration Parameters ===")
params = [
    "sm.compute_concurrency_level",
    "sm.io_concurrency_level",
    "sm.memory_budget",
    "vfs.s3.region",
    "vfs.s3.max_parallel_ops",
]
for param in params:
    try:
        value = tdb_config[param]
        print(f"  {param}: {value}")
    except KeyError:
        print(f"  {param}: (not set)")

=== TileDB Configuration Parameters ===
  sm.compute_concurrency_level: 4
  sm.io_concurrency_level: 4
  sm.memory_budget: 1073741824
  vfs.s3.region: us-east-1
  vfs.s3.max_parallel_ops: 8


### Advanced: Direct TileDB Context Creation

For specialized scenarios (isolated testing, custom thread pools), create contexts directly:

In [7]:
import tiledb

# Create a custom TileDB context with specific settings
custom_config = tiledb.Config(
    {
        "sm.compute_concurrency_level": "2",
        "sm.io_concurrency_level": "2",
        "sm.memory_budget": str(512 * 1024 * 1024),  # 512 MB
    }
)
custom_ctx = tiledb.Ctx(custom_config)

# Use custom context with RadiObject classes
test_data = np.random.randn(64, 64, 32).astype(np.float32)
uri = str(Path(TEMP_DIR) / "custom_ctx_vol")

# Volume accepts ctx parameter for custom context injection
vol = Volume.from_numpy(uri, test_data, ctx=custom_ctx)
print(f"Created volume with custom context: {vol}")

Created volume with custom context: Volume(shape=64x64x32, dtype=float32)


## Tile Orientations

Choose tile orientation based on your primary access pattern:

| Orientation | Tile Shape | Best For |
|-------------|------------|----------|
| `AXIAL` | 240 x 240 x 1 | Slice-by-slice viewing (radiology review) |
| `SAGITTAL` | 1 x 240 x 155 | Sagittal plane analysis |
| `CORONAL` | 240 x 1 x 155 | Coronal plane analysis |
| `ISOTROPIC` | 64 x 64 x 64 | 3D ROI extraction (ML training) |

```
    AXIAL (XY slices)          ISOTROPIC (64^3 cubes)
    +--------------+           +--------------+
    |==============| Z=0       | +--+--+--+   |
    |==============| Z=1       | +--+--+--+   |
    |==============| Z=2       | +--+--+--+   |
    |      ...     |           |    ...       |
    +--------------+           +--------------+
    
    Reading Z=77:              Reading 64^3 ROI:
    Reads 1 tile (fast)        Reads 1-8 tiles
```

In [8]:
# See how tile extents are computed for different orientations
shape = (240, 240, 155)

for orient in SliceOrientation:
    tile_cfg = TileConfig(orientation=orient)
    extents = tile_cfg.extents_for_shape(shape)
    print(f"{orient.value:10s} -> tile extents: {extents}")

axial      -> tile extents: (240, 240, 1)
sagittal   -> tile extents: (1, 240, 155)
coronal    -> tile extents: (240, 1, 155)
isotropic  -> tile extents: (64, 64, 64)


In [9]:
# Create test volumes with different tile orientations
test_data = np.random.randn(240, 240, 155).astype(np.float32)

volumes = {}
for orient in [SliceOrientation.AXIAL, SliceOrientation.SAGITTAL, SliceOrientation.ISOTROPIC]:
    configure(write=WriteConfig(tile=TileConfig(orientation=orient)))
    uri = str(Path(TEMP_DIR) / f"vol_{orient.value}")
    volumes[orient.value] = Volume.from_numpy(uri, test_data)
    print(f"Created {orient.value}: {volumes[orient.value]}")

Created axial: Volume(shape=240x240x155, dtype=float32)


Created sagittal: Volume(shape=240x240x155, dtype=float32)


Created isotropic: Volume(shape=240x240x155, dtype=float32)


In [10]:
# Benchmark: read axial slices from each volume
n_reads = 50
results = {}

for name, vol in volumes.items():
    start = time.perf_counter()
    for z in range(0, 150, 3):  # Read every 3rd slice
        _ = vol.axial(z)
    elapsed = time.perf_counter() - start
    results[name] = elapsed
    print(f"{name:10s}: {elapsed * 1000:.1f}ms for {n_reads} axial reads")

print(f"\nAxial-tiled is {results['isotropic'] / results['axial']:.1f}x faster for axial reads")

axial     : 117.5ms for 50 axial reads


sagittal  : 1157.4ms for 50 axial reads


isotropic : 630.5ms for 50 axial reads

Axial-tiled is 5.4x faster for axial reads


## Compression Options

| Compressor | Speed | Ratio | Best For |
|------------|-------|-------|----------|
| `NONE` | Fastest | 1.0x | Debugging, uncompressed formats |
| `LZ4` | Very fast | 1.0-1.2x | Real-time processing |
| `ZSTD` | Fast | 1.1-1.3x | **Default** - balanced |
| `GZIP` | Slower | 1.1-1.4x | Archival, max compression |

In [11]:
# Compare compression sizes
test_data = np.random.randn(120, 120, 60).astype(np.float32)
uncompressed_size = test_data.nbytes

for compressor in [Compressor.NONE, Compressor.LZ4, Compressor.ZSTD, Compressor.GZIP]:
    configure(
        write=WriteConfig(
            compression=CompressionConfig(algorithm=compressor, level=3),
            tile=TileConfig(orientation=SliceOrientation.AXIAL),
        )
    )
    uri = str(Path(TEMP_DIR) / f"vol_{compressor.value}")
    vol = Volume.from_numpy(uri, test_data)

    # Get directory size
    total_size = sum(
        os.path.getsize(os.path.join(dp, f))
        for dp, dn, filenames in os.walk(uri)
        for f in filenames
    )
    ratio = uncompressed_size / total_size if total_size > 0 else 0
    print(f"{compressor.value:6s}: {total_size / 1024:.1f} KB (ratio: {ratio:.2f}x)")

none  : 3382.6 KB (ratio: 1.00x)


lz4   : 3396.8 KB (ratio: 0.99x)
zstd  : 3128.4 KB (ratio: 1.08x)


gzip  : 3143.6 KB (ratio: 1.07x)


## Read Configuration

Read settings affect all data access operations:

| Parameter | Default | Description |
|-----------|---------|-------------|
| `memory_budget_mb` | 1024 | TileDB memory budget for I/O buffers |
| `concurrency` | 4 | TileDB internal thread pool size |
| `max_workers` | 4 | Python ThreadPoolExecutor workers |

In [12]:
# Configure for high-memory workload
configure(
    read=ReadConfig(
        memory_budget_mb=2048,  # 2GB memory budget
        concurrency=8,  # 8 parallel I/O threads
        max_workers=4,  # 4 parallel volume operations
    )
)

config = get_radiobject_config()
print(f"Memory budget: {config.read.memory_budget_mb} MB")
print(f"I/O concurrency: {config.read.concurrency} threads")
print(f"Max workers: {config.read.max_workers}")

Memory budget: 2048 MB
I/O concurrency: 8 threads
Max workers: 4


## S3 Configuration

RadiObject works identically with local and S3 URIs:

```python
# Local
radi = RadiObject("/data/study")

# S3 (same API!)
radi = RadiObject("s3://bucket/study")
```

Configure S3 for optimal performance:

In [13]:
configure(
    s3=S3Config(
        region="us-east-2",
        max_parallel_ops=16,  # Parallel S3 requests
        multipart_part_size_mb=50,  # Chunk size for uploads
    )
)

print("S3 configuration:")
print(f"  Region: {get_radiobject_config().s3.region}")
print(f"  Max parallel ops: {get_radiobject_config().s3.max_parallel_ops}")
print(f"  Multipart size: {get_radiobject_config().s3.multipart_part_size_mb} MB")

S3 configuration:
  Region: us-east-2
  Max parallel ops: 16
  Multipart size: 50 MB


## Configuration Recipes

### Radiology Viewer (2D Slice Access)

In [14]:
# Optimal for slice-by-slice viewing
reset_radiobject_config()
configure(
    write=WriteConfig(
        tile=TileConfig(orientation=SliceOrientation.AXIAL),
        compression=CompressionConfig(algorithm=Compressor.LZ4),  # Fast decompression
    ),
    read=ReadConfig(memory_budget_mb=512),  # Lower memory for interactive use
)
print("Configured for radiology viewer")

Configured for radiology viewer


### ML Training (3D Patch Extraction)

In [15]:
# Optimal for random 3D patch access
reset_radiobject_config()
configure(
    write=WriteConfig(
        tile=TileConfig(orientation=SliceOrientation.ISOTROPIC),
        compression=CompressionConfig(algorithm=Compressor.ZSTD, level=3),
    ),
    read=ReadConfig(
        memory_budget_mb=2048,  # More memory for training
        concurrency=8,
    ),
)
print("Configured for ML training")

Configured for ML training


### S3 Cloud Storage (High Bandwidth)

In [16]:
# Optimal for high-bandwidth S3 instances (p4d, p5)
reset_radiobject_config()
configure(
    read=ReadConfig(
        max_workers=8,  # More parallel volume operations
        concurrency=2,  # Fewer threads per operation
    ),
    s3=S3Config(
        max_parallel_ops=32,  # Maximize S3 parallelism
        multipart_part_size_mb=100,  # Larger parts for bandwidth
    ),
)
print("Configured for high-bandwidth S3")

Configured for high-bandwidth S3


In [17]:
# Cleanup
reset_radiobject_config()
shutil.rmtree(TEMP_DIR)
print(f"Cleaned up: {TEMP_DIR}")

Cleaned up: /var/folders/dj/0_0s64j55hn0gk7rrvj09zf80000gn/T/config_tutorial_y79k8qzb


## Summary

| Function | Purpose |
|----------|--------|
| `configure(write=, read=, s3=)` | Set RadiObject configuration |
| `get_radiobject_config()` | Get current RadiObjectConfig |
| `reset_radiobject_config()` | Reset to defaults |
| `get_tiledb_ctx()` | Get TileDB context (for advanced use) |
| `get_tiledb_config()` | Get raw TileDB Config |

### Quick Reference

```python
from radiobject import (
    configure, get_radiobject_config, reset_radiobject_config, get_tiledb_ctx, get_tiledb_config,
    WriteConfig, ReadConfig, S3Config,
    TileConfig, CompressionConfig,
    SliceOrientation, Compressor,
)

# Configure for slice viewing
configure(write=WriteConfig(tile=TileConfig(orientation=SliceOrientation.AXIAL)))

# Configure for ML training
configure(write=WriteConfig(tile=TileConfig(orientation=SliceOrientation.ISOTROPIC)))

# Inspect current config
print(get_radiobject_config().write.tile.orientation)

# Reset to defaults
reset_radiobject_config()

# Access TileDB internals
ctx = get_tiledb_ctx()     # For passing to TileDB operations
cfg = get_tiledb_config()     # For inspecting raw parameters
```